In [ ]:
%run ./time_series_encoding.py

synthetic_lf().collect().write_parquet("synthetic.parquet")
lf = load_and_preprocess_data("synthetic.parquet")
lf.head().collect()

Exception: File `'predictables/time_series_encoding.py'` not found.

In [ ]:
lf1 = recode_columns(lf)

lf1.head().collect()

index,feature1_1,feature1_2,feature1_3,feature1_4,feature1_5,feature1_6,feature1_7,feature1_8,feature1_9,feature1_10,feature1_11,feature1_12,feature1_13,feature1_14,feature1_15,feature1_16,feature1_17,feature1_18
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,39.965675,37.965675,35.965675,33.965675,31.965675,29.965675,27.965675,25.965675,23.965675,21.965675,19.965675,17.965675,15.965675,13.965675,11.965675,9.965675,7.965675,3.660958
1,36.273492,34.273492,32.273492,30.273492,28.273492,26.273492,24.273492,22.273492,20.273492,18.273492,16.273492,14.273492,12.273492,10.273492,8.273492,6.273492,4.273492,1.313476
2,41.618858,39.618858,37.618858,35.618858,33.618858,31.618858,29.618858,27.618858,25.618858,23.618858,21.618858,19.618858,17.618858,15.618858,13.618858,11.618858,9.618858,4.868407
3,42.442557,40.442557,38.442557,36.442557,34.442557,32.442557,30.442557,28.442557,26.442557,24.442557,22.442557,20.442557,18.442557,16.442557,14.442557,12.442557,10.442557,5.501992
4,34.240056,32.240056,30.240056,28.240056,26.240056,24.240056,22.240056,20.240056,18.240056,16.240056,14.240056,12.240056,10.240056,8.240056,6.240056,4.240056,2.240056,0.191092


In [ ]:
lf1.select([f"feature1_{i}" for i in range(1, 2 + 1)]).head().collect()

In [12]:
xy = X_y_generator(lf1, "feature1", 5, 2)
count = 0
for x, y in xy:
    count += 1
    print(f"count: {count}")
    print(
        f"\n\n==\nx:\n==\n\n{x.reshape(-1, 1)[:5]}\n\n==\ny:\n==\n\n{y.reshape(-1, 1)[:5]}"
    )

count: 1


==
x:
==

[[29.9656754 ]
 [26.27349209]
 [31.61885781]
 [32.44255692]
 [24.24005649]]

==
y:
==

[[31.9656754 ]
 [28.27349209]
 [33.61885781]
 [34.44255692]
 [26.24005649]]
count: 2


==
x:
==

[[31.9656754 ]
 [28.27349209]
 [33.61885781]
 [34.44255692]
 [26.24005649]]

==
y:
==

[[33.9656754 ]
 [30.27349209]
 [35.61885781]
 [36.44255692]
 [28.24005649]]
count: 3


==
x:
==

[[33.9656754 ]
 [30.27349209]
 [35.61885781]
 [36.44255692]
 [28.24005649]]

==
y:
==

[[35.9656754 ]
 [32.27349209]
 [37.61885781]
 [38.44255692]
 [30.24005649]]
count: 4


==
x:
==

[[35.9656754 ]
 [32.27349209]
 [37.61885781]
 [38.44255692]
 [30.24005649]]

==
y:
==

[[37.9656754 ]
 [34.27349209]
 [39.61885781]
 [40.44255692]
 [32.24005649]]
count: 5


==
x:
==

[[37.9656754 ]
 [34.27349209]
 [39.61885781]
 [40.44255692]
 [32.24005649]]

==
y:
==

[[39.9656754 ]
 [36.27349209]
 [41.61885781]
 [42.44255692]
 [34.24005649]]


In [44]:
def extract_features_and_lags(lf: pl.LazyFrame) -> tuple:
    features = [c.split("_")[2] for c in lf.select(cs.starts_with("logit")).columns]

    lags = [
        int(c.split("_")[-1].replace("]", ""))
        for c in lf.select(cs.starts_with("logit")).columns
    ]

    lf = lf.select(
        [pl.col("index")]
        + [
            pl.col(c).alias(f"{f}_{int(l/30)}")
            for c, f, l in zip(
                lf.select(cs.starts_with("logit")).columns, features, lags
            )
        ]
    )
    return lf, features, lags


df, feats, lags = extract_features_and_lags(lf)
df.head().collect()

index,feature1_1,feature1_2,feature1_3,feature1_4,feature1_5,feature1_6,feature1_7,feature1_8,feature1_9,feature1_10,feature1_11,feature1_12,feature1_13,feature1_14,feature1_15,feature1_16,feature1_17,feature1_18
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,39.965675,37.965675,35.965675,33.965675,31.965675,29.965675,27.965675,25.965675,23.965675,21.965675,19.965675,17.965675,15.965675,13.965675,11.965675,9.965675,7.965675,3.660958
1,36.273492,34.273492,32.273492,30.273492,28.273492,26.273492,24.273492,22.273492,20.273492,18.273492,16.273492,14.273492,12.273492,10.273492,8.273492,6.273492,4.273492,1.313476
2,41.618858,39.618858,37.618858,35.618858,33.618858,31.618858,29.618858,27.618858,25.618858,23.618858,21.618858,19.618858,17.618858,15.618858,13.618858,11.618858,9.618858,4.868407
3,42.442557,40.442557,38.442557,36.442557,34.442557,32.442557,30.442557,28.442557,26.442557,24.442557,22.442557,20.442557,18.442557,16.442557,14.442557,12.442557,10.442557,5.501992
4,34.240056,32.240056,30.240056,28.240056,26.240056,24.240056,22.240056,20.240056,18.240056,16.240056,14.240056,12.240056,10.240056,8.240056,6.240056,4.240056,2.240056,0.191092
